In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [63]:
df = pd.read_excel('ITU DATA.xlsx')
df1 = df[df['seriesCode'].isin(['i91'])]
#ICTTelephoneLinesPer100 = 391_i91
#ICTBroadbandSubscribersPer100ITU = 19303_i992b
#ICTTelephoneCellSubscribersPer100 = 392_i911
#ICTBroadbandMobileSubsPer100 = 12066_i911mw
#ICTInternetHousehold% = 12047_xHH6_IDI
#ICTComputerHousehold% = 12046_xHH4_IDI
# first part of code before _ is seriesID, second part is seriesCode

df2 = df1.drop(columns=['seriesID', 'seriesCode', 'seriesName', 'seriesParent', 'seriesUnits', 'entityID', 'entityIso', 'dataNote', 'dataSource', 'seriesDescription'])
df2 = df1.pivot_table(index = ["entityName"],
                    columns='dataYear',
                    values='dataValue')

#country concordance 
country = pd.read_excel('ITU Country.xlsx')
dt = country.merge(df2, on='entityName', how='left')
dt = dt.drop(columns='entityName')
dt = dt.rename(columns={'IFs Country':'Country'})
dt = dt.rename(columns={'dataYear':'Year'})


#dt = dt.reset_index()
dt.columns = [int(col) if str(col).replace('.', '', 1).isdigit() else col for col in dt.columns]

year_columns = [col for col in dt.columns if isinstance(col, int)]


dt['Earliest'] = dt[year_columns].bfill(axis=1).iloc[:, 0]
dt['MostRecent'] = dt[year_columns].ffill(axis=1).iloc[:, -1]

#change decimal place according to the datadict
dt[year_columns + ['Earliest', 'MostRecent']] = dt[year_columns + ['Earliest', 'MostRecent']].round(4)
dt

,Country,FIPS_CODE,1990,1991,1992,1993,1994,1995,1996,1997,...,2016,2017,2018,2019,2020,2021,2022,2023,Earliest,MostRecent
0,Afghanistan,AFG,0.3005,0.3023,0.2184,0.1941,0.1785,0.1699,0.1633,0.1572,...,0.3291,0.3328,0.3478,0.3557,0.3732,0.3645,0.3998,0.4389,0.3005,0.4389
1,Albania,ALB,1.2203,1.2642,1.3663,1.3118,1.2693,1.2935,1.9672,2.6876,...,8.5801,8.5128,8.5906,8.4180,7.7811,6.9374,6.2684,6.1578,1.2203,6.1578
2,Algeria,DZA,3.1999,3.3982,3.6136,3.9157,4.0248,4.1318,4.4024,4.7342,...,8.3345,9.8370,9.8833,10.7073,10.8663,11.3873,12.2615,13.6998,3.1999,13.6998
3,Angola,AGO,0.6021,0.5988,0.3936,0.4155,0.4002,0.3850,0.3714,0.4248,...,1.0434,0.5327,0.5491,0.3852,0.3562,0.3475,0.2638,0.2357,0.6021,0.2357
4,Argentina,ARG,9.2403,9.4228,10.6683,11.6607,14.4074,16.2936,17.6344,18.9909,...,23.1544,22.0018,21.8655,17.2485,16.2776,16.8307,15.5058,15.4468,9.2403,15.4468
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,"Venezuela, Bolivarian Republic",VEN,7.5034,7.8747,8.8142,9.7995,10.7443,11.0984,11.7679,12.1218,...,24.8992,19.3969,18.6103,18.4923,18.4614,11.4119,9.5112,8.4448,7.5034,8.4448
184,Viet Nam,VNM,0.1504,0.2050,0.2237,0.3736,0.6238,1.0756,1.6217,1.7957,...,5.9553,4.6077,4.4643,3.7644,3.2686,3.1561,2.3983,2.3082,0.1504,2.3082
185,Yemen,YEM,0.8966,0.9074,0.9566,1.0352,1.0708,1.1150,1.1824,1.2324,...,3.6308,3.5962,NaN,3.5316,3.4316,1.9639,1.9042,NaN,0.8966,1.9042
186,Zambia,ZMB,0.8355,0.8637,0.9280,0.9311,0.9303,0.8738,0.8656,0.8377,...,0.5995,0.5816,0.5588,0.5224,0.3769,0.3362,0.4778,0.3921,0.8355,0.3921


In [61]:
#sql table; change table name each time you import 
conn = sqlite3.connect(r'C:\IFs\RUNFILES\IFsDataImport.db')
cursor = conn.cursor()
table_name = 'SeriesICTComputerHousehold%'
#ICTDevelopmentIndexITU = ICT Development Index (IDI)
#ICTTelephoneLinesPer100 = 391_i91
#ICTBroadbandSubscribersPer100ITU = 19303_i992b
#ICTTelephoneCellSubscribersPer100 = 392_i911
#ICTBroadbandMobileSubsPer100 = 12066_i911mw
#ICTInternetHousehold% = 12047_xHH6_IDI
#ICTComputerHousehold% = 12046_xHH4_IDI


columns_sql = ['Country VARCHAR(255)', 'FIPS_CODE VARCHAR(255)']
columns_sql += [f'"{col}" DOUBLE(53)' for col in year_columns + ['Earliest', 'MostRecent']]
create_table_sql = f"CREATE TABLE '{table_name}' ({', '.join(columns_sql)})"
cursor.execute(f'DROP TABLE IF EXISTS "{table_name}"')
cursor.execute(create_table_sql)

dt.to_sql(table_name, conn, if_exists='append', index=False)

conn.commit()
conn.close()